In [32]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
} 

<IPython.core.display.Javascript object>

In [33]:
#File:StateStats.ipynb
#Author: Rafer Cooley
#Desc:state-wide comparisons of crime statistics

#https://plot.ly/python/table-subplots/
#https://towardsdatascience.com/time-series-analysis-in-python-an-introduction-70d5a5b1d52a
from ipywidgets import widgets
from ipywidgets import interactive
from IPython.display import display, HTML, clear_output
#from pivottablejs import pivot_ui
import matplotlib.pyplot as plt
import csv
import numpy as np
import pandas as pd
import os, sys
import subprocess
import base64 #for createdownloadlink

import plotly as pl
import plotly.figure_factory as ff
import plotly.graph_objs as go

import pandas as pd

pl.offline.init_notebook_mode(connected=True)

%run getdata.ipynb

my_styles = """
<style>
@media screen{
.container { width:90% !important; }
.page-break {page-break-after:always;}
}
@media print{
.container { width:100% !important; }
.page-break {page-break-after:always;}
}
</style>
"""
display(HTML(my_styles))

In [34]:
## MASTER VARIABLES
YEAR = '2013'
BEGIN_YEAR = '2012'
END_YEAR = '2017'

In [35]:
dfunct = DataFunctions()
overview = dfunct.getOverview()
dfs = dfunct.getDFS()#afcars, county, plc

agegroup_demographic_data = dfunct.getDemographic_By_AgeGroup()
population_data = dfunct.getPopulationData()#phasing out?
#arrest_totals = dfunct.getStateTotalArrests(juvenile_arrests,population_data)

school = dfunct.getSchool()
school_county_rates = dfunct.getSchool_county_rates(school,agegroup_demographic_data)
school_state_rates = dfunct.getSchool_state_rates(school,agegroup_demographic_data)

juvenile_arrests = dfunct.getORIData(agegroup_demographic_data)
arrest_totals = dfunct.getStateTotalArrests(juvenile_arrests,agegroup_demographic_data)

judicial_district_population_data = dfunct.getJudicialDistrictJuvenilePopulations(agegroup_demographic_data)
placement_data = dfunct.getJudicialPlacementData()
placement_rates_data,state_placement_rates_data = dfunct.getJudicialPlacementRates(placement_data,judicial_district_population_data)

demographic_data = dfunct.getDemographicData()
court_case_counts = dfunct.getCourtCaseNumbersData()

#qualifiers for ORI data
violent_felony = ['Manslaughter','Rape','Robbery','Aggravated-Assault']
lesser_offenses = ['Burglary','Larceny-Theft','Motor-Vehicle-Theft','Arson',
    'Other-Assaults','Forgery-Counterfeiting','Fraud','Embezzlement','Stolen-Property','Vandalism','Weapons',
    'Prostitution','Sex-Offenses','Drug-Abuse-Violations','Drug-Abuse-Violations-Possession','Gambling-Offenses',
    'Offenses-Against-Family-Children','Driving-Under-the-Influence','Liquor-Laws','Drunkenness',
    'Disorderly-Conduct','All-Other-Offenses-Except-Traffic','Suspicion','Curfew-Loitering-Law-Violations',
    'Run-a-ways','Vagrancy']

#qualifiers for placements data
placement_county_districts = []
placement_county_districts.append(['Laramie'])
placement_county_districts.append(['Albany','Carbon'])
placement_county_districts.append(['Uinta','Sweetwater','Lincoln'])
placement_county_districts.append(['Johnson','Sheridan'])
placement_county_districts.append(['Big Horn','Park','Hot Springs','Washakie'])
placement_county_districts.append(['Campbell','Weston','Crook'])
placement_county_districts.append(['Natrona'])
placement_county_districts.append(['Converse','Niobrara','Goshen','Platte'])
placement_county_districts.append(['Teton','Fremont','Sublette'])

#######################################################################
counties = ['Albany','Big Horn','Campbell','Carbon','Converse', 'Crook',
            'Fremont', 'Goshen', 'Hot Springs', 'Johnson', 'Laramie', 'Lincoln',
           'Natrona','Niobrara','Park','Platte','Sheridan','Sublette',
            'Sweetwater','Teton','Uinta','Washakie','Weston']
fips = [str(x) for x in range(56000,56046) if x%2!=0]
county_fips = [(counties[x]+' County',fips[x]) for x in range(len(counties))]
# print(str(county_fips))
# print(len(county_fips))
# print(len(counties))
# print(county_fips[0][0])

# colorscale = [
#     'rgb(63.0, 188.0, 115.0)',
#     'rgb(216.0, 226.0, 25.0)',
#     'rgb(38.0, 130.0, 142.0)',
#     'rgb(66.0, 64.0, 134.0)',
#     'rgb(68.0, 1.0, 84.0)'
# ]
    
colorscale = [
    'rgb(68.0, 1.0, 84.0)',
    'rgb(66.0, 64.0, 134.0)',
    'rgb(38.0, 130.0, 142.0)',
    'rgb(216.0, 226.0, 25.0)',
    'rgb(63.0, 188.0, 115.0)'
]
oppo_colorscale = ["#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce","#4292c6","#3082be","#2171b5","#1361a9",
              "#08519c","#0b4083","#08306b",
                  '#4d1300',
                  '#802000',
                  '#cc3300',
                  '#ff4000',
                  '#ff6633',
                  '#ff9f80',
                  '#ffc6b3'
                  ]
grey_scale = [x for x in range(0,255,int(255/25))]
grey_scale = ['rgb({},{},{})'.format(x,x,x) for x in grey_scale]
grey_scale.reverse()
#print(grey_scale)
#11 64 131
#8 48 107

display(HTML('<h1>State County Choropleth Graphs</h1>'))
display(HTML("""
<p>
The following graphs illustrate placement, educational discipline, and juvenile arrest rates broken up by county and/or judicial district as depicted on a map of Wyoming counties. 
The graphs represent lower rates as a brighter shade of white and counties with higher rates are portrayed as a darker shade of black.
</p>
"""))

In [36]:
########placement rates chloropleth#########
display(HTML('<h1>DFS Placement Rates for {}</h1>'.format(YEAR)))
district_num = 0
placement_values = []
for county in counties:
    for county_arr_idx in range(len(placement_county_districts)):#search through county list to match county name with judicial district number
        if county in placement_county_districts[county_arr_idx]:
            placement_values.append(placement_rates_data.loc[idx[county_arr_idx+1,YEAR:YEAR], :]['Rate'][0])
            #print('county:{} found in {}, len(vals)={}, with value of {}'.format(county,placement_county_districts[county_arr_idx],len(values),placement_rates_data.loc[idx[county_arr_idx+1,YEAR:YEAR], :]['Rate']))
            #district_num = county_arr_idx
            
display(HTML(placement_rates_data.loc[idx[:,YEAR:YEAR], :].transpose().to_html()))

#values = [x for x in range(23)]
#fips = [str(x) for x in range(56000,56046) if x%2!=0]
# print(len(values))
# print(len(fips))
# print(fips)
# print(values)

fig = ff.create_choropleth(
    fips=fips, values=placement_values,colorscale=grey_scale,
    scope=['Wyoming'], county_outline={'color': 'rgb(0,0,0)', 'width': 0.75},
    legend_title='Disctrict Rate'
)
fig['layout']['legend'].update({'x': -.2,'orientation':'h'})
fig['layout']['annotations'][0].update({'x': -0.12, 'xanchor': 'left'})
pl.offline.iplot(fig, filename='placement_rates_choropleth_wyoming')


district,1,2,3,4,5,6,7,8,9
Year,2013,2013,2013,2013,2013,2013,2013,2013,2013
Total,767.00,221.00,868.00,180.00,281.00,498.00,411.00,393.00,291.00
Population 11-17,8323.00,3452.00,8757.00,3162.00,4802.00,6027.00,7002.00,3449.00,6081.00
Rate,921.54,640.21,991.21,569.26,585.17,826.28,586.98,1139.46,478.54


In [37]:
########juvenile arrest rates chloropleth#########
display(HTML('<h1>Juvenile Arrest Rates for {}</h1>'.format(YEAR)))
display(HTML(juvenile_arrests[['Total Incidents','Population 11-17','Rate']].loc[idx[:,YEAR:YEAR], :].transpose().to_html()))
arrest_values = []
for county in counties:
    #print('county: {} with value: {}'.format(county,juvenile_arrests[['Total Incidents','Population 11-17','Rate']].loc[idx[county,YEAR:YEAR], :]['Rate'][0]))
    arrest_values.append(juvenile_arrests[['Total Incidents','Population 11-17','Rate']].loc[idx[county,YEAR:YEAR], :]['Rate'][0])
# print(len(values))
# print(len(fips))
# print(fips)
# print(values)

fig = ff.create_choropleth(
    fips=fips, values=arrest_values,colorscale=grey_scale,
    scope=['Wyoming'], county_outline={'color': 'rgb(0,0,0)', 'width': 0.75},
    legend_title='County Arrest Rate'
)
fig['layout']['legend'].update({'x': -.2,'orientation':'h'})
fig['layout']['annotations'][0].update({'x': -0.12, 'xanchor': 'left'})
# print(dir(fig['layout']['legend']))
# print(str(fig['layout']['legend']))
pl.offline.iplot(fig, filename='arrest_rates_choropleth_wyoming')

County,Albany,Big Horn,Campbell,Carbon,Converse,Crook,Fremont,Goshen,Hot Springs,Johnson,Laramie,Lincoln,Natrona,Niobrara,Park,Platte,Sheridan,Sublette,Sweetwater,Teton,Uinta,Washakie,Weston
Year,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013
Total Incidents,117.00,2.00,471.00,471.0,471.00,9.00,337.00,337.0,35.00,37.00,849.00,67.00,945.00,8.00,141.00,43.00,227.00,31.00,279.0,22.00,128.00,22.00,16.00
Population 11-17,2070.00,1231.00,4830.00,1382.0,1393.00,629.00,3730.00,1133.0,390.00,678.00,8323.00,1914.00,7002.00,204.00,2323.00,719.00,2484.00,904.00,4474.0,1447.00,2369.00,858.00,568.00
Rate,565.22,16.25,975.16,3408.1,3381.19,143.08,903.49,2974.4,897.44,545.72,1020.06,350.05,1349.61,392.16,606.97,598.05,913.85,342.92,623.6,152.04,540.31,256.41,281.69


In [38]:
########educational discipline rates chloropleth#########
display(HTML('<h1>Educational Discipline Rates for {}</h1>'.format(YEAR)))
display(HTML(school_county_rates[['Total','Population 11-17','Rate']].loc[idx[:,YEAR:YEAR], :].reset_index(level=1,drop=True).transpose().to_html()))
edu_values = []
for county in counties:
    #print('county: {} with value: {}'.format(county,juvenile_arrests[['Total Incidents','Population 11-17','Rate']].loc[idx[county,YEAR:YEAR], :]['Rate'][0]))
    edu_values.append(school_county_rates.loc[idx[county,YEAR:YEAR], :]['Rate'][0])
# print(len(values))
# print(len(fips))
# print(fips)
# print(values)

fig = ff.create_choropleth(
    fips=fips, values=edu_values,colorscale=grey_scale,
    scope=['Wyoming'], county_outline={'color': 'rgb(0,0,0)', 'width': 0.75},
    legend_title='County Incident Rate'
)
fig['layout']['legend'].update({'x': -.2,'orientation':'h'})
fig['layout']['annotations'][0].update({'x': -0.12, 'xanchor': 'left'})
# print(dir(fig['layout']['legend']))
# print(str(fig['layout']['legend']))
pl.offline.iplot(fig, filename='educational_discipline_rates_choropleth_wyoming')

County,Albany,Big Horn,Campbell,Carbon,Converse,Crook,Fremont,Goshen,Hot Springs,Johnson,Laramie,Lincoln,Natrona,Niobrara,Park,Platte,Sheridan,Sublette,Sweetwater,Teton,Uinta,Washakie,Weston
Total,67.00,64.0,122.00,228.00,41.00,10.00,673.00,78.00,7.00,15.00,105.00,21.00,1094.00,1.00,94.00,44.00,96.00,84.0,390.0,70.00,139.00,32.00,1.00
Population 11-17,2070.00,1231.0,4830.00,1382.00,1393.00,629.00,3730.00,1133.00,390.00,678.00,8323.00,1914.00,7002.00,204.00,2323.00,719.00,2484.00,904.0,4474.0,1447.00,2369.00,858.00,568.00
Rate,323.67,519.9,252.59,1649.78,294.33,158.98,1804.29,688.44,179.49,221.24,126.16,109.72,1562.41,49.02,404.65,611.96,386.47,929.2,871.7,483.76,586.75,372.96,17.61


In [39]:
# 5 year analysis
# display(HTML('<h2>5 Year State Analysis'))
# display(HTML('<p>A description of the producut goes here</p>'))
marker_colors = ['#6699ff','#00ff99','#669900','#ffcc00','#ff6600','#ff6600']
def generate_rate_data_obj(df,begin_year, end_year):
    years = [x for x in range(int(begin_year),int(end_year))]

    #gather sums arrays for each year
    sum_total = []
    for year in years:
        sum_total.append(df[['Rate']].loc[idx[:,year:year], :].values.flatten())
    #print(sum_total)
    #gather counties array
    counties = df.loc[idx[:,year:year], :].index.values.tolist()
    counties = [x[0] for x in counties]
    #print(counties)

    #CREATE RATE GRAPH
    data = []
    
    cntr = 0
    for year in years:
        year_avg = np.mean(sum_total[cntr])
        data.append(go.Bar(
            x = counties,
            y = sum_total[cntr],
            text=str(year),
            textposition = 'auto',
            name=year,
            marker=dict(
                color=marker_colors[cntr],
                line=dict(
                    color='rgb(8,48,107)',
                    width=1.5),
            ),
            opacity=0.6
            )
        )
        data.append(go.Scatter(
                x = counties,
                y = [year_avg for x in range(len(counties))],
                name='{} Average'.format(year),
                marker=dict(
                color=marker_colors[cntr],
                line=dict(
                    color='rgb(8,48,107)',
                    width=1.5),
            )
            )
        )
    
        cntr+=1
    return data

def generate_pct_change_data_obj(df,begin_year, end_year, district=False):
    years = [x for x in range(int(begin_year),int(end_year))]
    if not district:
        groups = counties
    else:
        groups = [x for x in range(1,10)]
        
    objs = []
    data = []
    for county in groups:
        pct_change_vals=df[['Rate']].loc[idx[county,begin_year:end_year], :].pct_change().values.flatten().tolist()
        objs.append({'county':county,'pct_change':pct_change_vals})
        cnt=0
        pos_years = []
        pos_data = []
        neg_years = []
        neg_data = []
        for year in years:
            if pct_change_vals[cnt]<0:
                neg_years.append(year)
                neg_data.append(pct_change_vals[cnt])
            else:
                pos_years.append(year)
                pos_data.append(pct_change_vals[cnt])
            cnt+=1
    #print(objs)

    cntr=0
    for year in years:
        year_avg = np.mean([x['pct_change'][cntr] for x in objs])
        data.append(go.Bar(
            x = groups,
            y = [x['pct_change'][cntr] for x in objs],
            #text=,
            textposition = 'auto',
            name='{} Percent Change'.format(year),
            marker=dict(
                color=marker_colors[cntr],
                line=dict(
                    color='rgb(8,48,107)',
                    width=1.5),
            ),
            opacity=0.6
            )
        )

        data.append(go.Scatter(
                x = groups,
                y = [year_avg for x in range(len(groups))],
                name='{} Average'.format(year),
                marker=dict(
                color=marker_colors[cntr],
                line=dict(
                    color='rgb(8,48,107)',
                    width=1.5),
            )
            )
        )

        cntr+=1
    return data

display(HTML('<h1>Statewide 5-year Comparison Charts</h1>'))
display(HTML("""
<p>
The following graphs illustrate  a 5 year comparison placement, educational discipline, and juvenile arrest rates broken up by county and/or judicial district.
</p>
"""))

In [40]:
display(HTML('<h2>School Discipline Rates Across All Counties {}-{}</h2>'.format(BEGIN_YEAR,END_YEAR)))
display(HTML("""
<p>
The graph below shows the number of educational discipline events(excluding in-school suspension) for each county
for the time period between {}-{}. Each county is represented along the x-axis, with 5 vertical bars for each 
county representing the educational discipline rate of that county for a particular year(vertical bars are 
color coded in the legen on the right). This discipline rate is calculated as the number of incidents 
per 10,000 residents.The horizontal lines across the graph represent the average discipline rate for the
entire state for a particular year(also color coded to correspond with year).
</p>
""".format(BEGIN_YEAR,END_YEAR)))

data = generate_rate_data_obj(school_county_rates,BEGIN_YEAR,END_YEAR)
pl.offline.iplot(data, filename='county school rates over 5 years')

In [41]:
display(HTML('<h2>Percent Change of School Discipline Rates Across All Counties {}-{}</h2>'.format(BEGIN_YEAR,END_YEAR)))
display(HTML("""
<p>
The graph below shows the percent change in educational displine rates from the previous year(i.e. 2013 
shows how much the rate has changed from 2012). Each county is represented along the x-axis, with 5 vertical bars for each 
county representing the educational discipline rate of that county for a particular year. If the bar extends below 
the x-axis then the county experienced decrease in educational discipline incidents, and vice-versa for bars extending 
above the x-axis. The horizontal bars on this graph represent the average percentage of change in incidents for 
the entire state, with lines above the x-axis representing an overall increase in incidents across the state for that
particular year. Similiar to the previous graphs the bars and lines are color coded according to year as represented 
by the legend on the right.
</p>
""".format(BEGIN_YEAR,END_YEAR)))

data = generate_pct_change_data_obj(school_county_rates,BEGIN_YEAR,END_YEAR)
pl.offline.iplot(data, filename='county school pct change in rates over 5 years')

In [42]:
#print(school_county_rates[['Rate']].loc[idx[:,'2012':'2017'], :].reset_index().head(3))

#school_county_rates['Percent Change']=school_county_rates[['Rate']].loc[idx[:,:], :].groupby('County').pct_change()
#print(school_county_rates['Percent Change'].loc[idx[:,'2012':'2017'], :])

In [43]:
display(HTML('<h2>Juvenile Arrest Rates Across All Counties {}-{}</h2>'.format(BEGIN_YEAR,END_YEAR)))

data = generate_rate_data_obj(juvenile_arrests,BEGIN_YEAR,END_YEAR)
pl.offline.iplot(data, filename='county arrests rates over 5 years')

In [44]:
display(HTML('<h2>Percent Change of Juvenile Arrest Rates Across All Counties {}-{}</h2>'.format(BEGIN_YEAR,END_YEAR)))
years = [x for x in range(int(BEGIN_YEAR),int(END_YEAR))]
for year in years:
    print(juvenile_arrests[['Rate']].loc[idx[county,year:year], :])
          #.values.flatten().tolist())


#data = generate_pct_change_data_obj(juvenile_arrests,BEGIN_YEAR,str(int(END_YEAR)))
    
#pl.offline.iplot(data, filename='county arrests rates over 5 years')

               Rate
County Year        
Weston 2012  812.18
               Rate
County Year        
Weston 2013  281.69
               Rate
County Year        
Weston 2014  644.63
               Rate
County Year        
Weston 2015  561.06
               Rate
County Year        
Weston 2016  260.59


In [45]:
display(HTML('<h2>DFS Placement Rates Across All Counties {}-{}</h2>'.format(BEGIN_YEAR,END_YEAR)))

print(placement_rates_data.head(3))
#print(state_placement_rates_data[['Rate']].loc[idx[:,year:year], :].values.flatten())
#print(state_placement_rates_data[['Rate']].loc[idx[:,year:year], :].values.flatten())

data = generate_rate_data_obj(placement_rates_data,BEGIN_YEAR,END_YEAR)
pl.offline.iplot(data, filename='county arrests rates over 5 years')

               Total  Population 11-17    Rate
district Year                                 
1        2012    449            8193.0  548.03
         2013    767            8323.0  921.54
         2014    690            8374.0  823.98


In [46]:
display(HTML('<h2>Percent Change of DFS Placement Rates Across All Counties {}-{}</h2>'.format(BEGIN_YEAR,END_YEAR)))
#print(juvenile_arrests[['Rate']].loc[idx[county,BEGIN_YEAR:END_YEAR], :].pct_change().values.flatten().tolist())

data = generate_pct_change_data_obj(placement_rates_data,BEGIN_YEAR,str(int(END_YEAR)),district=True)
pl.offline.iplot(data, filename='county arrests rates over 5 years')

In [47]:
# data = [
#     go.Scatter(
#         x = ['2016','2017','2018'],
#         y = [500,600,700],
#     ),
#     go.Bar(
#         x = ['2016','2017','2018'],
#         y = [500,600,700],
#         base = [-500,-600,-700],
#         marker = dict(
#           color = 'red'
#         ),
#         name = 'expenses'
#     ),
#     go.Bar(
#         x = ['2016','2017','2018'],
#         y = [300,400,700],
#         base = 0,
#         marker = dict(
#           color = 'blue'
#         ),
#         name = 'revenue'
#     )
# ]


# fig = go.Figure(data=data)
# pl.offline.iplot(fig, filename='base-bar')

In [48]:
# import re 

# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/Mining-BTC-180.csv')

# # remove min:sec:millisec from dates 
# for i, row in enumerate(df['Date']):
#     p = re.compile(' 00:00:00')
#     datetime = p.split(df['Date'][i])[0]
#     df.iloc[i, 1] = datetime
    
# table_trace2 = go.Table(
#     domain=dict(x=[0, 1],
#                 y=[0.7, 1.0]),  
#     columnwidth=[1, 2, 2, 2],
#     columnorder=[0, 1, 2, 3, 4],
#     header = dict(height = 50,
#                   values = [['<b>Date</b>'],['<b>Hash Rate, TH/sec</b>'], 
#                             ['<b>Mining revenue</b>'], ['<b>Transaction fees</b>']], 
#                   line = dict(color='rgb(50, 50, 50)'),
#                   align = ['left'] * 5,
#                   font = dict(color=['rgb(45, 45, 45)'] * 5, size=14),
#                   fill = dict(color='#d562be')),
#     cells = dict(values = [df[k].tolist() for k in ['Date', 'Hash-rate', 'Mining-revenue-USD', 'Transaction-fees-BTC']],
#                  line = dict(color='#506784'),
#                  align = ['left'] * 5,
#                  font = dict(color=['rgb(40, 40, 40)'] * 5, size=12),
#                  format = [None] + [", .2f"] * 2 + [',.4f'],  
#                  prefix = [None] * 2 + ['$', u'\u20BF'],
#                  suffix=[None] * 4,
#                  height = 27,
#                  fill = dict(color=['rgb(235, 193, 238)', 'rgba(228, 222, 249, 0.65)']))
# )

# trace4=go.Scatter(
#     x=df['Date'],
#     y=df['Hash-rate'],
#     xaxis='x1',
#     yaxis='y1',
#     mode='lines',
#     line=dict(width=2, color='#9748a1'),
#     name='hash-rate-TH/s'
# )

# trace5=go.Scatter(
#     x=df['Date'],
#     y=df['Mining-revenue-USD'],
#     xaxis='x2',
#     yaxis='y2',
#     mode='lines',
#     line=dict(width=2, color='#b04553'),
#     name='mining revenue'
# )

# trace6=go.Scatter(
#     x=df['Date'],
#     y=df['Transaction-fees-BTC'],
#     xaxis='x3',
#     yaxis='y3',
#     mode='lines',
#     line=dict(width=2, color='#af7bbd'),
#     name='transact-fee'
# )

# axis=dict(
#     showline=True,
#     zeroline=False,
#     showgrid=True,
#     mirror=True, 
#     ticklen=4, 
#     gridcolor='#ffffff',
#     tickfont=dict(size=10)
# )

# layout2 = dict(
#     width=950,
#     height=800,
#     autosize=False,
#     title='Bitcoin mining stats for 180 days',
#     margin = dict(t=100),
#     showlegend=False,          
#     xaxis1=dict(axis, **dict(domain=[0, 1], anchor='y1', showticklabels=False)),
#     xaxis2=dict(axis, **dict(domain=[0, 1], anchor='y2', showticklabels=False)),          
#     xaxis3=dict(axis, **dict(domain=[0, 1], anchor='y3')), 
#     yaxis1=dict(axis, **dict(domain=[2 * 0.21 + 0.02 + 0.02, 0.68], anchor='x1', hoverformat='.2f')),  
#     yaxis2=dict(axis, **dict(domain=[0.21 + 0.02, 2 * 0.21 + 0.02], anchor='x2', tickprefix='$', hoverformat='.2f')),
#     yaxis3=dict(axis, **dict(domain=[0.0, 0.21], anchor='x3', tickprefix=u'\u20BF', hoverformat='.2f')),
#     plot_bgcolor='rgba(228, 222, 249, 0.65)'
# )

# fig2 = dict(data=[table_trace2, trace4, trace5, trace6], layout=layout2)
# pl.offline.iplot(fig2, filename='vertical-stacked-subplot-tables')

In [49]:
# vals = [['<b>{}</b>'.format(county)] for county in counties]
# print(vals)

# table_trace2 = go.Table(
#     domain=dict(x=[0, 1],
#                 y=[0.7, 1.0]),  
#     columnwidth=[2 for x in range(len(vals))],
#     columnorder=[x for x in range(len(vals))],
#     header = dict(height = 50,
#                   values = vals, 
#                   line = dict(color='rgb(50, 50, 50)'),
#                   align = ['left'] * 5,
#                   font = dict(color=['rgb(45, 45, 45)'] * 5, size=14),
#                   fill = dict(color='#d562be')),
#     cells = dict(values = [df[k].tolist() for k in ['Date', 'Hash-rate', 'Mining-revenue-USD', 'Transaction-fees-BTC']],
#                  line = dict(color='#506784'),
#                  align = ['left'] * 5,
#                  font = dict(color=['rgb(40, 40, 40)'] * 5, size=12),
#                  format = [None] + [", .2f"] * 2 + [',.4f'],  
#                  prefix = [None] * 2 + ['$', u'\u20BF'],
#                  suffix=[None] * 4,
#                  height = 27,
#                  fill = dict(color=['rgb(235, 193, 238)', 'rgba(228, 222, 249, 0.65)']))
# )

# fig2 = dict(data=[table_trace2])
# pl.offline.iplot(fig2, filename='vertical-stacked-subplot-tables')

In [50]:
# ########educational discipline rates chloropleth#########
# display(HTML('<h1>Educational Discipline Rates for {}</h1>'.format(YEAR)))
# #display(HTML(school_county_rates[['Total','Population','Rate']].loc[idx[:,YEAR:YEAR], :].reset_index(level=1,drop=True).transpose().head(3).to_html()))
# edu_values = []
# for county in counties:
#     #print('county: {} with value: {}'.format(county,juvenile_arrests[['Total Incidents','Population 11-17','Rate']].loc[idx[county,YEAR:YEAR], :]['Rate'][0]))
#     edu_values.append(school_county_rates.loc[idx[county,YEAR:YEAR], :]['Rate'][0])

# vals = [['<b>{}</b>'.format(county)] for county in counties]
# #print(vals)

# edu_df = school_county_rates[['Total','Population','Rate']].loc[idx[:,YEAR:YEAR], :].reset_index(level=1,drop=True).transpose()
# #print(edu_df.head(2))
# # print(edu_df.index)
# # print(edu_df.loc[idx['Rate', :]])
# totals = edu_df.loc[idx['Total', :]].tolist()
# totals.insert(0,'Totals Incidents')
# pops = edu_df.loc[idx['Population', :]].tolist()
# pops.insert(0,'Population')
# rates = edu_df.loc[idx['Rate', :]].tolist()
# rates.insert(0,'Rate')

# heads = edu_df.columns.tolist()
# heads.insert(0,'Type')
# print(heads)
# table_trace2 = go.Table(
#     domain=dict(x=[0, 1],
#                 y=[0.7, 1.0]),  
#     columnwidth=[2 for x in range(len(vals))],
#     columnorder=[x for x in range(len(vals))],
#     header = dict(height = 50,
#                   values = heads, 
#                   line = dict(color='rgb(50, 50, 50)'),
#                   align = ['left'] * 5,
#                   font = dict(color=['rgb(45, 45, 45)'] * 5, size=14),
#                   fill = dict(color='#d562be')),
#     cells = dict(
#                  values = [[p,t,r] for p,t,r in zip(pops,totals,rates)],
#                  line = dict(color='#506784'),
#                  align = ['left'] * 5,
#                  font = dict(color=['rgb(40, 40, 40)'] * 5, size=12),
#                  #height = 27,
#                  fill = dict(color=['rgb(235, 193, 238)', 'rgba(228, 222, 249, 0.65)']))
# )

# data = [table_trace2]
# # print(dir(fig['layout']['legend']))
# # print(str(fig['layout']['legend']))
# pl.offline.iplot(data, filename='educational_discipline_rates_choropleth_wyoming')

In [51]:
#EXAMPLE
# values = [x for x in range(23)]
# #fips = [str(x) for x in range(56000,56046) if x%2!=0]
# # print(len(values))
# # print(len(fips))

# fig = ff.create_choropleth(
#     fips=fips, values=values,
#     scope=['Wyoming'], county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
#     legend_title='Population per county'
   
# )
# fig['layout']['legend'].update({'x': 0})
# fig['layout']['annotations'][0].update({'x': -0.12, 'xanchor': 'left'})
# pl.offline.iplot(fig, filename='choropleth_wyoming')

In [52]:
# district_num = 0
# for county_arr_idx in range(1,len(placement_county_districts)):#search through county list to match county name with judicial district number
#     if county in placement_county_districts[county_arr_idx]:
#         district_num = county_arr_idx
# display(HTML(placement_rates_data.loc[idx[district_num+1,year_tup[0]:year_tup[1]], :].transpose().to_html()))

# display(HTML(state_placement_rates_data.loc[idx[year_tup[0]:year_tup[1]], :].transpose().to_html()))

# display(HTML(demographic_data.loc[idx[county,:,year_tup[0]:year_tup[1]],:].transpose().to_html()))

# display(HTML(juvenile_arrests[['Total Incidents','Population 11-17','Rate']].loc[idx[county,year_tup[0]:year_tup[1]], :].transpose().to_html()))
  
# display(HTML(arrest_totals[['Total Incidents','Population 11-17','Rate']].loc[idx[year_tup[0]:year_tup[1]],:].transpose().to_html()))

